<a href="https://colab.research.google.com/github/Seboldju/OCR-Stromzaehler-Arbeitsdokument/blob/main/Extraktion_Bilder_aus_Collagen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import os
import zipfile
import shutil

# Laden der Collage (Datei wurde zuvor in Collab hochgeladen)
image_path = '/content/1-s2.0-S235248471930174X-gr7_lrg.jpg'
large_image = cv2.imread(image_path)

# Überprüfen, ob die Collage erfolgreich geladen wurde
if large_image is None:
    raise FileNotFoundError(f"Das Bild konnte nicht geladen werden. Überprüfe den Pfad: {image_path}")

# Konvertierung des Bildes in Graustufen
gray_image = cv2.cvtColor(large_image, cv2.COLOR_BGR2GRAY)

# Thresholding, um die Stromzählerbilder vom Hintergrund zu trennen
_, thresh = cv2.threshold(gray_image, 240, 255, cv2.THRESH_BINARY_INV)

# Auffinden der Konturen der Stromzählerbilder
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Erstellung eines Verzeichnisses (extracted_images), zum Speichern der einzelnen Bilder
output_dir = 'extracted_images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Extraktion und Speicherung jedes gefundenen Bildes im Ordner extracted_images
for i, contour in enumerate(contours):
    x, y, w, h = cv2.boundingRect(contour)
    cropped_image = large_image[y:y+h, x:x+w]
    output_path = os.path.join(output_dir, f'meter_{i+1}.jpg')
    cv2.imwrite(output_path, cropped_image)
    print(f'Bild {i+1} gespeichert: {output_path}')

print('Alle Bilder wurden extrahiert und gespeichert.')

# Erstellen einer ZIP-Datei aus dem Ordner
zip_file = 'extracted_images.zip'
with zipfile.ZipFile(zip_file, 'w') as zipf:
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_dir))

print(f'ZIP-Datei erstellt: {zip_file}')

# Sicherstellung, dass der Zielpfad existiert
target_dir = '/mnt/data/'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Kopieren der ZIP-Datei in das Verzeichnis, das zum Herunterladen verwendet wird
shutil.copy(zip_file, target_dir + zip_file)

print(f'Die ZIP-Datei wurde zum Herunterladen vorbereitet: {target_dir}{zip_file}')

Bild 1 gespeichert: extracted_images/meter_1.jpg
Bild 2 gespeichert: extracted_images/meter_2.jpg
Bild 3 gespeichert: extracted_images/meter_3.jpg
Bild 4 gespeichert: extracted_images/meter_4.jpg
Bild 5 gespeichert: extracted_images/meter_5.jpg
Bild 6 gespeichert: extracted_images/meter_6.jpg
Alle Bilder wurden extrahiert und gespeichert.
ZIP-Datei erstellt: extracted_images.zip
Die ZIP-Datei wurde zum Herunterladen vorbereitet: /mnt/data/extracted_images.zip


Anschließend: Download der zip-Datei und händische Bereinigung der Bilder (es wurden einige zusätzliche Bilder erkannt, die gar nicht existierten). So wurden erfolgreich aus 5 Collagen 5 Ordner mit einzelnen Bilddateien von Stromzählern extrahiert